In [ ]:
import json
from collections import defaultdict
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import random
import cv2
from pycocotools.coco import COCO

with open("/opt/ml/dataset/train.json",'r') as f:
    json_data = json.load(f)
print(type(json_data))
print(json.dumps(json_data,indent="\t"))

coco = COCO("/opt/ml/dataset/train.json")

In [ ]:
print(json_data.keys())
print("---------------categories--------------")
for i in json_data['categories']:
    print(i['name'])

In [ ]:
df = pd.DataFrame({'image_id':[], "category_id":[], 'bbox_x':[], 'bbox_y':[], 'bbox_w':[], 'bbox_h':[], 'ratio':[], 'area':[], 'iscrowd':[], 'id':[]})
for i,info in enumerate(json_data['annotations']):
    df.loc[i] = [
        int(info['image_id']), 
        int(info['category_id']), 
        info['bbox'][0], 
        info['bbox'][1],
        info['bbox'][2],
        info['bbox'][3],
        round(info['bbox'][2]/info['bbox'][3],1),
        info['area'],
        int(info['iscrowd']),
        info['id']
        ]
df['image_id'] = df['image_id'].astype('int32')
df['category_id'] = df['category_id'].astype('int32')
df['iscrowd'] = df['iscrowd'].astype('int32')
df['id'] = df['id'].astype('int32')
print(df.head())

In [ ]:
"""
하나의 이미지 안에 Bounding Box가 몇개씩 들어있는지를 나타내는 분포입니다.
"""
# 각 이미지의 박스 개수 계산
box_counts = df.groupby('image_id').size()
print(f"max : {max(box_counts)}")

# 히스토그램 그리기
plt.figure(figsize=(10, 6))
plt.hist(box_counts, bins=70, edgecolor='black',align='left')
plt.xlabel('Box Count')
plt.ylabel('Frequency')
plt.title('Box Count Per Image')
plt.show()

In [ ]:
"""
Bounding Box가 하나의 이미지에 많이 분포해있는 경우를 눈으로 확인하기 위한 EDA입니다.
아래 TODO를 수정해서 threshold와 몇개의 이미지를 plot 할 것인지 정할 수 있습니다.
"""
# TODO: 아래 두개의 변수를 변경하면 됩니다.
threshold = 3 # bounding box 몇개 포함된 이미지들을 뽑을건지
max_plot = 5 # 몇 개의 이미지를 plot할 것인지 (5배수)

CATEGORY_COLOR = [(204, 0, 0), (0, 0, 153),
                  (204, 204, 0), (51, 51, 51),
                  (230, 230, 230), (75, 0, 130),
                  (255, 102, 0), (255, 105, 180),
                  (77, 77, 77), (0, 102, 0)]
# 각 이미지의 박스 개수 계산
box_counts = df.groupby('image_id').size()
# Series 객체를 Data frame으로 변환
box_counts_df = box_counts.reset_index(name='box_count')
print(box_counts_df)

# 박스 수가 10인 이미지들의 리스트 생성
selected_images = box_counts_df[box_counts_df['box_count'] == threshold]['image_id'].tolist()
print(selected_images)

# 리스트에서 랜덤으로 10개 선택
random_images = random.sample(selected_images, min(len(selected_images), max_plot))

# 선택된 이미지들을 subplot에 시각화
fig, axs = plt.subplots(max_plot//5, 5, figsize=(20, max_plot), constrained_layout=True)
axs = axs.ravel()

image_folder = "/opt/ml/dataset"
# 서브플롯에 이미지와 어노테이션 정보 표시
# for i, ax in enumerate(axs.flat):
#     # i번째 이미지와 어노테이션 정보 가져오기
#     img_id = random_images[i]
#     image_name = json_data['images'][image_id]['file_name']
#     img_path = f'{image_folder}/{image_name}'
#     ann_ids = coco.getAnnIds(imgIds=img_id)
#     anns = coco.loadAnns(ann_ids)

#     # 이미지와 어노테이션 정보를 서브플롯에 표시
#     image = cv2.imread(img_path)[:, :, ::-1]
#     ax.imshow(image)
#     print(anns)
#     coco.showAnns(anns, draw_bbox=True)
    
for i, image_id in enumerate(random_images):
    image_name = json_data['images'][image_id]['file_name']
    image = cv2.imread(f'{image_folder}/{image_name}')
    x, y, w, h = df[df['image_id'] == image_id]['bbox_x'], df[df['image_id'] == image_id]['bbox_y'], df[df['image_id'] == image_id]['bbox_w'], df[df['image_id'] == image_id]['bbox_h']
    # 이미지와 어노테이션 정보 가져오기
    ann_ids = coco.getAnnIds(imgIds=image_id)
    anns = coco.loadAnns(ann_ids)
    for ann in anns:
        x, y, w, h = map(int, ann['bbox'])
        category = ann['category_id']
        cv2.rectangle(image, (x, y), (x+w, y+h), CATEGORY_COLOR[category], 8)
        font = cv2.FONT_HERSHEY_SIMPLEX
        font_scale = 1.5
        cv2.putText(image, json_data['categories'][category]['name'], (x, y-20), font, font_scale, CATEGORY_COLOR[category], thickness=5)

    # 어노테이션 정보 시각화
    axs[i].imshow(image)
    axs[i].set_title(f'Image ID: {image_id}')
    axs[i].axis('off')
    
plt.show()

In [ ]:
"""
클래스별로 bounding box들을 랜덤샘플링해서 plot했습니다.
"""
cls_boxes = df.groupby(['category_id', 'image_id'])['id'].apply(list)
ct_img_ids = df.groupby('category_id')['image_id'].unique().apply(list)
# print(cls_boxes)
# print(ct_img_ids)
fig, axs = plt.subplots(nrows=10, ncols=5, figsize=(20, 50), constrained_layout=True)
axs = axs.ravel()

# 10개 클래스에 대해 랜덤으로 이미지 5개 뽑아서 plot
for i in range(10):
    random_images = random.sample(ct_img_ids[i], min(len(ct_img_ids), 5))
    for j, img_id in enumerate(random_images):
        image_name = json_data['images'][img_id]['file_name']
        image = cv2.imread(f'{image_folder}/{image_name}')
        anns = coco.loadAnns(cls_boxes[i][img_id])
        # 각 클래스에 해당하는 어노테이션 정보 가져오기
        for ann in anns:
            x, y, w, h = map(int, ann['bbox'])
            category = ann['category_id']
            cv2.rectangle(image, (x, y), (x+w, y+h), CATEGORY_COLOR[category], 8)
            font = cv2.FONT_HERSHEY_SIMPLEX
            font_scale = 1.5
            cv2.putText(image, json_data['categories'][category]['name'], (x, y-20), font, font_scale, CATEGORY_COLOR[category], thickness=5)
        # 어노테이션 정보 시각화
        axs[i*5+j].imshow(image)
        axs[i*5+j].set_title(f'Image ID: {image_id}')
        axs[i*5+j].axis('off')
    
plt.show()

In [ ]:
"""
Bounding Box의 width, height의 크기에 따라 scatter plot으로 표현합니다.
즉 모든 이미지에 대한 Bounding Box의 크기와 가로세로 비율 분포를 나타낸 도표입니다.
"""
areas = df['area']
print(f"max : {max(areas)}")
print(f"min : {min(areas)}")

bboxes = list(zip(df['bbox_w'], df['bbox_h']))
colors = list(df['category_id'])
# 스캐터 플롯 그리기
plt.scatter([box[0] for box in bboxes], [box[1] for box in bboxes], s=3, c=colors, cmap='Spectral')
plt.colorbar()

plt.show()

In [ ]:
"""
각 클래스 대한 Bounding Box의 크기와 가로세로 비율 분포를 나타낸 도표입니다.
"""
bbox_w = df.groupby('category_id')['bbox_w'].apply(list)
bbox_h = df.groupby('category_id')['bbox_h'].apply(list)

fig, axs = plt.subplots(nrows=2, ncols=5, figsize=(20, 8), constrained_layout=True)
axs = axs.ravel()

for i in range(10):
    axs[i].scatter(bbox_w[i], bbox_h[i], s=3)
    axs[i].set_title(f"{i}. {json_data['categories'][i]['name']}")
    
plt.show()

In [ ]:
def iou(box1, box2):
    x = max(box1[0], box2[0])
    y = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])
    intersection_area = max(0, x2 - x1 + 1) * max(0, y2 - y1 + 1)
    box1_area = (box1[2] - box1[0] + 1) * (box1[3] - box1[1] + 1)
    box2_area = (box2[2] - box2[0] + 1) * (box2[3] - box2[1] + 1)
    union_area = box1_area + box2_area - intersection_area
    iou = intersection_area / union_area
    return iou


cls_boxes = df.groupby('category_id')['image_id'].reset_index(name='imgid')
cls_boxes
